In [1]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()
client = OpenAI()

In [8]:
# create if not exist
# vector_store = client.beta.vector_stores.create(
#     name="knowledge-repo"
# )
stores = client.beta.vector_stores.list()
vector_store = None
for store in stores.data:
    vector_store_id = store.id
    print(f"vector store {store.name}: {store.id}")
    if store.name == "knowledge-repo":
        vector_store = store
    # clean up if necessary
    #     client.beta.vector_stores.delete(store.id)

vector store knowledge-repo: vs_683f2a04c0748191a65d91c2f414c755
vector store Temporal: vs_ewunN7laFAOw6m8mEOF5jixY


In [7]:
def get_filtered_file_paths(directory, extensions):
    filtered_files = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.split('.')[-1].lower() in extensions:
                file_path = os.path.join(root, file)
                if os.path.getsize(file_path) == 0:
                    print('file is empty', file_path)
                else:
                    filtered_files.append(file_path)
    return filtered_files

def process_directory(subdir_path, vector_store):
    file_paths = get_filtered_file_paths(subdir_path, supported_extensions)
    if not file_paths:
        print(f"Skipping {subdir_path}: no valid files.")
        return

    print(f"Processing {subdir_path} with {len(file_paths)} files...")

    file_streams = [open(path, "rb") for path in file_paths]
    try:
        file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
            vector_store_id=vector_store.id, files=file_streams
        )
        print(f"{subdir_path} → {file_batch.status} | files: {file_batch.file_counts}")
        print(f"Vector Store ID: {vector_store.id}")
    finally:
        for f in file_streams:
            f.close()


# Define supported extensions
supported_extensions = {"md"}

# Top-level directory to search
root_dir = "/Users/hungcq/projects/knowledge-repo"

# Process each top-level subdirectory
for item in os.listdir(root_dir):
    subdir_path = os.path.join(root_dir, item)
    if os.path.isdir(subdir_path):
        process_directory(subdir_path, vector_store)

Processing /Users/hungcq/projects/knowledge-repo/practical with 13 files...
/Users/hungcq/projects/knowledge-repo/practical → completed | files: FileCounts(cancelled=0, completed=13, failed=0, in_progress=0, total=13)
Vector Store ID: vs_683f2a04c0748191a65d91c2f414c755
Processing /Users/hungcq/projects/knowledge-repo/biography with 4 files...
/Users/hungcq/projects/knowledge-repo/biography → completed | files: FileCounts(cancelled=0, completed=4, failed=0, in_progress=0, total=4)
Vector Store ID: vs_683f2a04c0748191a65d91c2f414c755
Processing /Users/hungcq/projects/knowledge-repo/art with 4 files...
/Users/hungcq/projects/knowledge-repo/art → completed | files: FileCounts(cancelled=0, completed=4, failed=0, in_progress=0, total=4)
Vector Store ID: vs_683f2a04c0748191a65d91c2f414c755
Processing /Users/hungcq/projects/knowledge-repo/psychology with 10 files...
/Users/hungcq/projects/knowledge-repo/psychology → completed | files: FileCounts(cancelled=0, completed=10, failed=0, in_progres

In [29]:
all_files = client.beta.vector_stores.files.list(vector_store_id=vector_store.id)

failed = [f for f in all_files if f.status != "completed"]
print(len(failed))